In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses. 

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
PROJECT_ID = "qwiklabs-gcp-03-b4b21a101e96"  # @param {type:"string"}
LOCATION = "us-east4"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [3]:
model = GenerativeModel("gemini-1.5-flash")

In [4]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [5]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop specializing in dried flowers, playing on different themes:

**Whimsical & Romantic:**

* Everlasting Blooms
* The Dried Flower Garden
* Whispers of Wildflowers
* The Petal Press
* Bloom & Botanicals
* The Dusty Bouquet
* Moonflower & Co.

**Modern & Minimalist:**

* The Bloom Bar
* Flora & Fauna
* Wild & Dried
* Paper Blooms
* The Dried Flower Collective
* Bloom Alchemy
* The Botanical Studio

**Rustic & Vintage:**

* The Dried Flower Barn
* The Old World Blooms
* The Herbarium
* The Vintage Bouquet
* Sun-Kissed Blooms
* The Dried Flower Market
* Rustic Blooms

**Playful & Unique:**

* Petal Pushers
* Bloom & Beyond
* The Flower Fix
* Dried & Dandy
* The Flower Apothecary
* Bloom & Co.
* The Flower Fairy

**Tips for Choosing a Name:**

* **Keep it short and easy to remember.**
* **Reflect your brand's personality and aesthetic.**
* **Consider your target audience.**
* **Check for availability of the name and domain name.**
* **Get feedback from fr

✅ Recommended. The prompt below is to the point and concise.

In [6]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Catchy & Chic:

* Bloom & Breathe
* The Dried Petal
* Everlasting Blooms
* Whispers of Bloom
* Flora Forever
* Petal & Ink
* Bloom & Bone
* The Dried Bouquet
* The Flower Archivist

## Rustic & Natural:

* Field & Fern
* The Wildflower Studio
* Sun-Kissed Blooms
* The Dried Garden
* Rustic Blooms
* The Harvest Bouquet
* The Meadow Muse
* Willow & Thistle

## Unique & Playful:

* The Flower Apothecary
* The Botanical Bar
* The Bloom Collective
* The Dried Dream
* Timeless Blooms
* Paper & Petal
* The Flower Library
* The Bloom Box

##  Name + Location:

* [Your Town] Dried Flowers
* The [Your Street] Flower Shop
* [Your Name] Dried Florals
* The [Your Neighborhood] Blooms

**Bonus Tip:** 
Consider including a tagline to further clarify your brand, such as: 
* "Dried Flowers that Last a Lifetime"
* "Bringing the Beauty of Nature Indoors"
* "Preserving the Moment, Forever"

**To choose the best name for your shop, think about:**

* Your target audience
* The overall aesthetic and feel 

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [7]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known celestial body to harbor life. Here's a glimpse into its wonders:

**Basics:**

* **Size:** Diameter of 12,742 km (7,926 miles)
* **Mass:** 5.972 × 10^24 kg 
* **Composition:** Primarily iron, nickel, silicon, oxygen, magnesium, sulfur, and aluminum
* **Age:** Approximately 4.54 billion years old

**Unique Features:**

* **Liquid Water:** Earth is the only planet in our solar system with abundant liquid water, crucial for life as we know it.
* **Atmosphere:** Our atmosphere, composed mainly of nitrogen and oxygen, protects us from harmful solar radiation and allows for weather patterns.
* **Plate Tectonics:** The Earth's crust is divided into plates that move and interact, leading to geological phenomena like earthquakes, volcanic eruptions, and mountain formation.
* **Magnetic Field:** Earth's magnetic field protects us from the Sun's harmful solar wind.
* **Biodiversity:** Earth boasts an incredible 

✅ Recommended. The prompt below is specific and well-defined.

In [8]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What makes Earth unique? 

Earth stands out from other planets in our solar system and likely beyond, thanks to a remarkable combination of factors:

**1. Liquid Water:**  Earth is the only known planet with abundant liquid water on its surface. This is crucial for life as we know it, serving as a solvent, transporting nutrients, and playing a vital role in numerous biological processes.

**2. Plate Tectonics:** Earth's constantly shifting tectonic plates create mountains, volcanoes, and earthquakes. This dynamic process also plays a role in regulating climate and recycling nutrients through the Earth's crust.

**3. Atmosphere:** Earth's atmosphere, rich in oxygen and nitrogen, protects us from harmful solar radiation, regulates temperature, and supports weather patterns.

**4. Magnetic Field:**  Earth's magnetic field acts as a shield against harmful solar wind, protecting life from radiation.

**5. Distance from the Sun:** Earth's orbit is located within the habitable zone, known 

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [9]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Let's tackle these two questions separately:

**Boiling Water**

The best method of boiling water depends on your specific needs and available resources. Here's a breakdown of common methods:

* **Stovetop:** This is the most common method for boiling water. Simply fill a pot with water, place it on a stove burner, and turn the heat to high. This method is efficient and works well for most situations.
* **Microwave:** You can boil water in a microwave-safe container. This method is faster than stovetop boiling, but it's important to use a container specifically designed for microwave use and avoid overheating.
* **Electric Kettle:** Electric kettles are designed specifically for boiling water. They are efficient and fast, and they often feature automatic shut-off features for safety. 
* **Camping Kettle:** If you're outdoors, a camping kettle can be a great option. They are lightweight and portable, and some models even have a built-in whistle to alert you when the water boils.

**Why 

✅ Recommended. The prompts below asks one task a time.

In [10]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

The "best" method of boiling water really depends on your needs and preferences. Here's a breakdown of common methods and their pros and cons:

**1. Stovetop:**

* **Pros:** Most versatile, allows for larger quantities, can be easily controlled.
* **Cons:** Can take longer than electric kettles, requires constant attention.
* **Best for:** When you need a large amount of boiling water or prefer the control it offers.

**2. Electric Kettle:**

* **Pros:** Fastest method, convenient, usually has automatic shut-off.
* **Cons:** Limited capacity, can be noisy.
* **Best for:** When speed is a priority and you only need a small amount of water.

**3. Microwave:**

* **Pros:** Quick, convenient, energy-efficient.
* **Cons:** Can be uneven heating, limited capacity, not suitable for all containers.
* **Best for:** Small quantities of water, when you need it quickly.

**4. Water Boiler:**

* **Pros:** Continuous hot water supply, can be used for other purposes like making tea.
* **Cons:** More 

In [11]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky is blue because of a phenomenon called **Rayleigh scattering**. Here's the breakdown:

* **Sunlight is made up of all colors of the rainbow.**  Think of a prism splitting sunlight into its colors.
* **The Earth's atmosphere is filled with tiny particles of gas.**  These particles are much smaller than the wavelengths of visible light.
* **When sunlight enters the atmosphere, it interacts with these particles.**  Shorter wavelengths of light, like blue and violet, are scattered more strongly than longer wavelengths, like red and orange.
* **This scattered blue light reaches our eyes from all directions, making the sky appear blue.**

**Why not violet?** While violet light is scattered even more strongly than blue, our eyes are more sensitive to blue.  

**Here are some additional factors:**

* **Time of day:** The sky appears more blue at midday because the sunlight travels through less atmosphere. At sunrise and sunset, the light travels through more atmosphere, scattering away

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [14]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To know what day it is, please check a calendar or your device's clock. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [15]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan is a fantastic city for sightseeing! To give you the best recommendation, I need a little more information.  What kind of sights are you interested in? Are you looking for historical landmarks, art museums, shopping, or something else entirely? 

Tell me more about your interests, and I can suggest some incredible places to visit in Milan! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [16]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm an AI chatbot for a travel website, so I can only help you with travel-related queries. 

Maybe you can ask me about popular restaurants in Milan, or if you have any other travel-related questions, I'd be happy to help! 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [17]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

Here are some programming activities tailored for high school students, aiming to improve skills in various ways:

**For beginners:**

* **Build a simple calculator:**  This project helps solidify basic syntax, arithmetic operations, and input/output.
* **Create a text-based adventure game:**  This encourages creative thinking, using conditional statements and loops.
* **Develop a simple guessing game:**  Practice using random number generation, loops, and conditional statements.
* **Design a basic to-do list:**  Learn about data structures (like lists or arrays) and user input.
* **Build a password checker:**  Explore string manipulation, conditional statements, and validation.

**For intermediate learners:**

* **Create a basic web page with HTML, CSS, and JavaScript:**  Learn about web development fundamentals and start building interactive websites.
* **Develop a simple Python script for automating tasks:**  Learn about automation and work with files and data.
* **Build a basic cha

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [18]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

As a high school student, all three programming languages have their own merits and can be valuable to learn. Here's a breakdown to help you choose:

**a) Learn Python:**

* **Pros:**
    * **Beginner-friendly:** Python has a simple syntax, making it easier for beginners to grasp.
    * **Versatile:** Widely used in web development, data science, machine learning, and more.
    * **Large community:** Extensive online resources, libraries, and support available.
    * **In-demand:**  Highly sought-after skill in various industries.

* **Cons:**
    * **Not as fast as some other languages:** Python can be slower than compiled languages for certain tasks.

**b) Learn JavaScript:**

* **Pros:**
    * **Interactive web development:**  Essential for creating dynamic and interactive web pages.
    * **Front-end and back-end:**  Can be used for both client-side and server-side development.
    * **Growing ecosystem:** Constantly evolving with new libraries and frameworks.
    * **High demand:*

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [19]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [20]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [21]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 

Here's why:

* **"Something surprised me"**: This indicates a positive experience. 
* **"It was actually original"**:  This is a compliment and suggests the video is good. 
* **"It was not the same old recycled stuff"**: This implies the video is refreshing and different, a positive attribute. 
* **"Watch it - you will not regret it"**: This is a strong recommendation, clearly positive. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.